In [7]:
import sqlite3
from datetime import datetime

def initialize_database(db_path="default.db"):
    """Initialize the database and create tables if they don't exist"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create calendar table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS calendar (
        date TEXT PRIMARY KEY,
        day INTEGER,
        day_of_week TEXT,
        month TEXT,
        year INTEGER
    )
    ''')
    
    # Create numbers table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS numbers (
        id TEXT PRIMARY KEY,
        date TEXT,
        number INTEGER,
        FOREIGN KEY(date) REFERENCES calendar(date)
    )
    ''')
    
    # Create stars table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS stars (
        id TEXT PRIMARY KEY,
        date TEXT,
        star INTEGER,
        FOREIGN KEY(date) REFERENCES calendar(date)
    )
    ''')
    
    # Create indexes for better performance
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_numbers_date ON numbers(date)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_stars_date ON stars(date)')
    
    conn.commit()
    conn.close()

def add_calendar_date(conn, date):
    """Add a date to the calendar table"""
    date_obj = datetime.strptime(date, '%Y-%m-%d')
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR IGNORE INTO calendar (date, day, day_of_week, month, year)
    VALUES (?, ?, ?, ?, ?)
    ''', (date, date_obj.day, date_obj.strftime('%A'), date_obj.strftime('%B'), date_obj.year))
    conn.commit()

def add_number(conn, date, number):
    """Add a number entry for a specific date"""
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR IGNORE INTO numbers (id, date, number)
    VALUES (?, ?, ?)
    ''', (date+str(number), date, number))
    conn.commit()
    return cursor.lastrowid

def add_star(conn, date, star):
    """Add a stars entry for a specific date"""
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR IGNORE INTO stars (id, date, star)
    VALUES (?, ?, ?)
    ''', (date+str(star), date, star))
    conn.commit()
    return cursor.lastrowid

def get_data_count(conn, per_weekday=0):
    """Retrieve all data with counts, if per_weekday then group by it"""
    cursor = conn.cursor()
      
    # Get numbers
    if per_weekday:
        cursor.execute("SELECT n.number FROM numbers as n, calendar c WHERE n.date = c.date AND c.day_of_week = 'Tuesday'")
        numbers_tuesday = [row[0] for row in cursor.fetchall()]  
        cursor.execute("SELECT n.number FROM numbers as n, calendar c WHERE n.date = c.date AND c.day_of_week = 'Friday'")
        numbers_friday = [row[0] for row in cursor.fetchall()]  
        numbers = [numbers_tuesday, numbers_friday]
        cursor.execute("SELECT s.star FROM stars as s, calendar c WHERE s.date = c.date AND c.day_of_week = 'Tuesday'")
        stars_tuesday = [row[0] for row in cursor.fetchall()]  
        cursor.execute("SELECT s.star FROM stars as s, calendar c WHERE s.date = c.date AND c.day_of_week = 'Friday'")
        stars_friday = [row[0] for row in cursor.fetchall()]
        stars = [stars_tuesday, stars_friday]
    else:
        cursor.execute('SELECT number FROM numbers')
        numbers = [row[0] for row in cursor.fetchall()]
        cursor.execute('SELECT star FROM stars')
        stars = [row[0] for row in cursor.fetchall()]    

    return numbers, stars